In [1]:
import pandas as pd
import geopandas as gpd
import os
from tqdm import tqdm

In [2]:
floods_alert_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\1_extract_flood_inDMA\flood_warning_alert\flood_warning_alert.csv"
DMA_shp_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\DMA\DMA shape file\DMA.shp"
floods_alert_shp_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\orignal\Flood_Alert_Areas (1)\Flood_Alert_AreasPolygon.shp"
DMA_mapping_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Anglian_water\processed\DMA_Mapping.csv"

In [3]:
DMA_shp = gpd.read_file(DMA_shp_path)
floods_alert_shp = gpd.read_file(floods_alert_shp_path)
DMA_shp=DMA_shp.to_crs(epsg = 3857)
floods_alert_shp = floods_alert_shp.to_crs(epsg=3857)

In [4]:
DMA_mapping = pd.read_csv(DMA_mapping_path)

In [5]:
floods_alert_shp['alert_area_ID'] = range(1,len(floods_alert_shp)+1)

In [6]:
overlap = gpd.overlay(floods_alert_shp,DMA_shp,how='intersection',keep_geom_type=False)
overlap = overlap.drop_duplicates().reset_index(drop = True)

In [7]:
dma_per_flood = overlap.groupby('alert_area_ID')['DMASHORTNA'].apply(list).reset_index(name='DMA_ids')
dma_per_flood 

,alert_area_ID,DMA_ids
0,1,"[THEALBMA, CHERRYMA, WRBRIGMA, NEWBARMA, CARYL..."
1,2,"[IWNLDYXX, CLAYPLMA, GRCNNOMA, ALLINGMA, SKILI..."
2,4,"[OLDLENMA, ALGAKIMA, FRISKNMA, WAINFLMA, SKEGS..."
3,6,"[BOSLEAMA, MARSHLMA, BOSTWTSS, BOPRIOMA, BOWHI..."
4,9,"[METWESMA, METEASMA, BRAEASMA, POTTERMA, BRAWE..."
...,...,...
118,241,[IPSRE5MA]
119,242,"[CRETNGMA, CHILHAMA, NEEDNOMA, ONEHSEMA, IWNLU..."
120,243,"[SEMEWWDV, SUDCNTMA, CAPSMNMA, SUDCOCMA, SUDCA..."
121,246,"[HORNNWMA, WASHBYMA, BILLHYMA, BELCHFMA, LUDFO..."


In [8]:
# Create a mapping dictionary from DMA_Name to DMA_ID
dma_name_to_id = dict(zip(DMA_mapping['DMA_Name'], DMA_mapping['DMA_ID']))

# Replace DMA names in the 'DMA_ids' column with their corresponding numerical IDs
dma_per_flood['DMA_ids'] = dma_per_flood['DMA_ids'].apply(
    lambda x: [dma_name_to_id[dma] for dma in x if dma in dma_name_to_id]
)

# Display the updated dataframe
print(dma_per_flood.head())

   alert_area_ID                                            DMA_ids
0              1  [1771, 364, 2002, 1202, 346, 330, 845, 1955, 1...
1              2  [381, 705, 14, 1556, 1587, 692, 678, 113, 693,...
2              4       [1329, 12, 636, 1854, 1553, 1328, 632, 1330]
3              6  [174, 1090, 171, 184, 168, 1542, 1135, 1729, 1...
4              9            [1116, 1114, 214, 1382, 241, 103, 1871]


In [9]:
floods_alert = pd.read_csv(floods_alert_path)

In [10]:
floods_alert['CODE'] = floods_alert['CODE'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else (np.nan if isinstance(x, list) else x)
)
floods_alert['CODE'] = floods_alert['CODE'].astype(str)
alert_code_to_id = floods_alert_shp[['fws_tacode','alert_area_ID']]
floods_alert = floods_alert.merge(alert_code_to_id, left_on='CODE', right_on='fws_tacode', how='left')
floods_alert.drop(columns=['fws_tacode'],inplace = True)
floods_alert = floods_alert.merge(dma_per_flood, left_on='alert_area_ID', right_on='alert_area_ID', how='left')
floods_alert['alert_ID'] = range(1,len(floods_alert)+1)
floods_alert

,DATE,AREA,CODE,WARNING / ALERT AREA NAME,TYPE,year,alert_area_ID,DMA_ids,alert_ID
0,2020/1/3 13:24,"East Anglia - Essex, Norfolk and Suffolk",054WATBT2,The tidal River Yare from Thorpe St Andrew to ...,Flood Alert,2020,154,"[1049, 1798, 1279, 1407, 1273, 1673, 628, 1674...",1
1,2020/1/3 13:39,"East Anglia - Essex, Norfolk and Suffolk",054WATBT3,The tidal River Waveney from Ellingham to Brey...,Flood Alert,2020,181,"[314, 1798, 1407, 1674, 1050]",2
2,2020/1/8 16:08,East Midlands,034WAF430,River Maun in Nottinghamshire,Flood Alert,2020,134,"[555, 1683, 539]",3
3,2020/1/8 17:41,West Midlands - East,033WAF201,Upper Avon River Swift and Clay Coton Brook,Flood Alert,2020,64,"[957, 499, 1913, 1540, 2028]",4
4,2020/1/8 17:53,West Midlands - East,033WAF203,River Leam and River Itchen,Flood Alert,2020,65,"[1627, 240, 810, 71]",5
...,...,...,...,...,...,...,...,...,...
3137,2024/10/21 9:45,"East Anglia - Essex, Norfolk and Suffolk",051WACDV4C,"The Essex coast from Clacton to and including,...",Flood Alert,2024,116,"[393, 394, 411]",3138
3138,2024/10/21 17:03,"East Anglia - Essex, Norfolk and Suffolk",054WATBT1,"The tidal Rivers Bure, Ant and Thurne",Flood Alert,2024,185,"[1613, 776, 851, 575, 1746, 1265, 1308, 853, 1...",3139
3139,2024/10/21 17:13,"East Anglia - Essex, Norfolk and Suffolk",054WATBT2,The tidal River Yare from Thorpe St Andrew to ...,Flood Alert,2024,154,"[1049, 1798, 1279, 1407, 1273, 1673, 628, 1674...",3140
3140,2024/10/21 17:19,"East Anglia - Essex, Norfolk and Suffolk",054WATBT3,The tidal River Waveney from Ellingham to Brey...,Flood Alert,2024,181,"[314, 1798, 1407, 1674, 1050]",3141


In [11]:
output_dir = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\2_extract_flood_date_waterflow\flood_alert"
water_flow_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Anglian_water\processed\DMA_flow"

In [12]:
floods_alert['DATE'] = pd.to_datetime(floods_alert['DATE'])

In [14]:
for _, flood_row in tqdm(floods_alert.iterrows(),total=len(floods_alert)):
    dma_ids = flood_row['DMA_ids']
    for dma_id in dma_ids:
        output_file_name = f"alert_{flood_row['alert_ID']}_{flood_row['DATE'].strftime('%Y%m%d')}_alertarea_{flood_row['alert_area_ID']}_dma_{dma_id}.csv"
        output_path = os.path.join(output_dir,output_file_name)

        if os.path.exists(output_path):
            continue
        
        dma_file = f"Flow_DMA_{dma_id}.csv"
        dma_file_path = os.path.join(water_flow_path,dma_file)
        dma_data = pd.read_csv(dma_file_path)
        dma_data['Date_Time'] = pd.to_datetime(dma_data['Date_Time'])
        
        expanded_startdate = flood_row['DATE'] - pd.Timedelta(days = 7)
        expanded_enddate = flood_row['DATE'] + pd.Timedelta(days = 7)
        extracted_data = dma_data[(dma_data['Date_Time']>=expanded_startdate) & (dma_data['Date_Time']<=expanded_enddate)]

        extracted_data.to_csv(output_path,index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3142/3142 [3:49:20<00:00,  4.38s/it]
